In [2]:
!pip install pandas pandasql openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from pandasql import sqldf as sql

In [4]:
df = pd.read_csv('electric.csv')
df.columns = df.columns.str.replace(' ', '_')

In [5]:
df

,VIN_(1-10),County,City,State,Postal_Code,Model_Year,Make,Model,Electric_Vehicle_Type,Clean_Alternative_Fuel_Vehicle_(CAFV)_Eligibility,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID,Vehicle_Location,Electric_Utility,2020_Census_Tract
0,5UXTA6C05P,Yakima,Yakima,WA,98903.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,14.0,227153587,POINT (-120.477805 46.553505),PACIFICORP,5.307700e+10
1,5YJRE11B48,NaN,NaN,BC,NaN,2008,TESLA,ROADSTER,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,98950,NaN,143609049,NaN,NaN,NaN
2,5YJSA1E24G,King,Seattle,WA,98103.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,43.0,187728201,POINT (-122.34301 47.659185),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1N4AZ1CP5J,King,Shoreline,WA,98177.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,32.0,249867971,POINT (-122.382425 47.77279),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303302e+10
4,5YJ3E1EA6J,Island,Coupeville,WA,98239.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,10.0,223792649,POINT (-122.6880708 48.2179983),PUGET SOUND ENERGY INC,5.302997e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153825,1N4AZ0CP0D,San Juan,Eastsound,WA,98245.0,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,40.0,154379130,POINT (-122.907229 48.7016716),BONNEVILLE POWER ADMINISTRATION||ORCAS POWER &...,5.305596e+10
153826,5YJSA1E27H,Spokane,Spokane,WA,99223.0,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,6.0,204991475,POINT (-117.369705 47.62637),BONNEVILLE POWER ADMINISTRATION||INLAND POWER ...,5.306301e+10
153827,1G1FW6S03P,King,Seatac,WA,98188.0,2023,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,33.0,244720059,POINT (-122.29179 47.43473),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
153828,3FMTK3SU1M,King,Kent,WA,98031.0,2021,FORD,MUSTANG MACH-E,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,33.0,186104215,POINT (-122.2012521 47.3931814),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10


In [6]:
sql('select distinct state from df order by 1')

,State
0,AK
1,AL
2,AP
3,AR
4,AZ
5,BC
6,CA
7,CO
8,CT
9,DC


In [15]:
#_join(cars, mounts, "a.*, b.MountName", "a.MountID = b.ID", "a.MountID", "b.MountName")
def _join(df1: pd.DataFrame, df2: pd.DataFrame, fields:str, on:str, a_key:str, b_key:str, generateLeft=True, generateRight=True):
    left, right = None, None
    if generateLeft:
        left = sql(f'select {fields} from df1 a left join df2 b on {on} where {b_key} is null')
    center = sql(f'select {fields} from df1 a inner join df2 b on {on}')
    if generateRight:
        right = sql(f'select {fields} from df1 a right join df2 b on {on} where {a_key} is null')
    return left, center, right

#_union_all(cars, ['mounts', 'colors'])
def _union_all(df1: pd.DataFrame, dfs:list[str]):
    return sql(f'select * from df1 union all {"union all ".join(["select * from "+dfx for dfx in dfs])}')

#_summarize(cars, ['ColorID', 'MountID'], {'price': 'mean'})
def _summarize(df1: pd.DataFrame, group:list[str], summ:dict):
    arr_summ = [f"{summ[summ_item]}({summ_item}) as {summ_item}" for summ_item in summ]
    return sql(f'select {", ".join(group)}, {", ".join(arr_summ)} from df1 group by {",".join([str(x) for x in list(range(1,len(group)+1))])}')

#_filter(cars, "ColorID = 1")
def _filter(df1: pd.DataFrame, conditional:str, generateFalse=True):
    t = sql(f'select * from df1 where ({conditional})')
    if generateFalse:
        f = sql(f'select * from df1 where not ({conditional})')
        return t, f
    return t, None

#_compare(cars, mounts)
def _compare(df1: pd.DataFrame, df2: pd.DataFrame):
    left, right = [], []
    
    l = [y.lower() for y in df1.columns.to_list()]
    r = [y.lower() for y in df2.columns.to_list()]
    
    for x in l:
        if x.lower() not in r:
            left.append(x)
    for x in r:
        if x.lower() not in l:
            right.append(x)
    
    return len(left) == 0 and len(right) == 0, left, right

states = pd.read_excel('states.xlsx')

In [8]:
t, f = _filter(df, "State='NY'", generateFalse=True)
f

,VIN_(1-10),County,City,State,Postal_Code,Model_Year,Make,Model,Electric_Vehicle_Type,Clean_Alternative_Fuel_Vehicle_(CAFV)_Eligibility,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID,Vehicle_Location,Electric_Utility,2020_Census_Tract
0,5UXTA6C05P,Yakima,Yakima,WA,98903.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,14.0,227153587,POINT (-120.477805 46.553505),PACIFICORP,5.307700e+10
1,5YJRE11B48,None,None,BC,NaN,2008,TESLA,ROADSTER,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,98950,NaN,143609049,None,None,NaN
2,5YJSA1E24G,King,Seattle,WA,98103.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,43.0,187728201,POINT (-122.34301 47.659185),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1N4AZ1CP5J,King,Shoreline,WA,98177.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,32.0,249867971,POINT (-122.382425 47.77279),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303302e+10
4,5YJ3E1EA6J,Island,Coupeville,WA,98239.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,10.0,223792649,POINT (-122.6880708 48.2179983),PUGET SOUND ENERGY INC,5.302997e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153818,1N4AZ0CP0D,San Juan,Eastsound,WA,98245.0,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,40.0,154379130,POINT (-122.907229 48.7016716),BONNEVILLE POWER ADMINISTRATION||ORCAS POWER &...,5.305596e+10
153819,5YJSA1E27H,Spokane,Spokane,WA,99223.0,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,6.0,204991475,POINT (-117.369705 47.62637),BONNEVILLE POWER ADMINISTRATION||INLAND POWER ...,5.306301e+10
153820,1G1FW6S03P,King,Seatac,WA,98188.0,2023,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,33.0,244720059,POINT (-122.29179 47.43473),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
153821,3FMTK3SU1M,King,Kent,WA,98031.0,2021,FORD,MUSTANG MACH-E,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,33.0,186104215,POINT (-122.2012521 47.3931814),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10


In [17]:
_compare(df, df)

(True, [], [])

In [14]:
_summarize(df, ["Make"], {"Make": "count"})

,Make,Make
0,ALFA ROMEO,18
1,AUDI,3094
2,AZURE DYNAMICS,8
3,BENTLEY,1
4,BMW,6620
5,CADILLAC,191
6,CHEVROLET,12242
7,CHRYSLER,2755
8,FIAT,797
9,FISKER,17


In [10]:
states

,State,Name
0,AL,Alabama
1,CA,California
2,NY,Nova Iorque
3,OH,Ohio


In [11]:
_,c,_ = _join(df, states, 'a.*, b.Name as State_Name','a.State = b.State', 'a.State', 'b.Name', generateLeft=False, generateRight=False)

In [12]:
l, len(c), len(r)

NameError: name 'l' is not defined

In [ ]:
sql('select distinct State from l order by 1')